<a href="https://colab.research.google.com/github/autinn/deep-tech-talent/blob/main/Lab06_H_3_convolutional_neural_network_Student_Version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab06: Create a Basic Convolutional Neural Network for Image Classification**

## 1. Import Library and Data

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#@title Code for loading the data
# import libraries
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt

# utilities library
import cv2
import os

import random
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)
os.environ['PYTHONHASHSEED'] = '0'


data_path = '/content/gdrive/My Drive/Colab Notebooks/data/jaffe/jaffe'
data_dir_list = os.listdir(data_path)

img_data_list=[]
img_label = []
label_map = {}
i = 0

for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    label_map[i] = str(dataset)

    # print ('Load the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        img_label.append(i)
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img_resize=cv2.resize(input_img,(128,128))
        img_data_list.append(input_img_resize)
    i += 1

img_data = np.array(img_data_list)
img_data = img_data.astype('float')
img_data = img_data/255 #normalization
img_label = np.array(img_label)

# num_classes = 6
# num_of_samples = img_data.shape[0]
# img_label = np.ones((num_of_samples,),dtype='int')

# img_label[0:29]=0 #30
# img_label[30:58]=1 #29
# img_label[59:90]=2 #32
# img_label[91:121]=3 #31
# img_label[122:152]=4 #31
# img_label[153:]=5 #30
# names = ['ANGRY','DISGUST','FEAR','HAPPY','SAD','SURPRISE']

def getlabel(id):
    return label_map[id]

**About this Data**:

We will use the Japanese Female Facial Expression (JAFFE) dataset which has 183 images of 10 different female models posing for 6 emotions. The data will be normalized so its value will ranging from 0 to 1.

The label will be denoted as numbers, which follow this mapping :

|    | Expression   |
|---:|:-------------|
|  0 | DISGUST          |
|  1 | SAD          |
|  2 | SURPRISE       |
|  3 |   ANGRY    |
|  4 |  HAPPY      |
|  5 |   FEAR    |

<details> <summary>Cite the author</summary>
Michael J. Lyons, Shigeru Akamatsu, Miyuki Kamachi, Jiro Gyoba. Coding Facial Expressions with Gabor Wavelets, 3rd IEEE International Conference on Automatic Face and Gesture Recognition, pp. 200-205 (1998). http://doi.org/10.1109/AFGR.1998.670949 Open access content available at: https://zenodo.org/record/3430156

</details>

split the data into train and test set so that we can perform **cross validation** later

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(img_data, img_label, test_size=0.1, random_state=0)

___

## 2. Create the Model

In previous two notebook, we already created a similar model using keras. Here's the code if you wanted to remember.

In [ ]:
# simple model in 1st Notebook
model_simple = keras.Sequential()
model_simple.add(keras.layers.Dense(units=1, input_shape=[1]))

model_simple.compile(optimizer='SGD', loss='mean_squared_error')

In [ ]:
# Neural Network in 2nd Notebook
model_nn = keras.Sequential()

model_nn.add(keras.layers.Flatten(input_shape=(128,128,3)) ) # flatten layer
model_nn.add(keras.layers.Dense(units = 128,activation='relu') )  #input layer
model_nn.add(keras.layers.Dense(units = 64, activation='relu') )  # hidden layer
model_nn.add(keras.layers.Dense(units = 6, activation='softmax') ) # output layer

model_nn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In this Lab, we will build a simple CNNs to do the image classification. We are now trying to add a convolutional part to our model. Mind the changes compared to previous basic neural network

In [ ]:
model_cnn = keras.Sequential()

model_cnn.add(keras.layers.Conv2D(filters=16, kernel_size=(5,5), input_shape=(128,128,3), padding='same', activation="relu"))
model_cnn.add(keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding='same', activation="relu"))
model_cnn.add(keras.layers.MaxPooling2D())

model_cnn.add(keras.layers.Flatten()) # flatten layer
model_cnn.add(keras.layers.Dense(units = 128,activation='relu') )  #input layer
model_cnn.add(keras.layers.Dense(units = 64, activation='relu') )  # hidden layer
model_cnn.add(keras.layers.Dense(units = 6, activation='softmax') ) # output layer

model_cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# summarize the model to see what happened inside
model_cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 16)      1216      
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      4640      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                                 
 dense_1 (Dense)             (None, 128)               16777344  
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                      

## 3. Train the Model

In [ ]:
model_cnn.fit(x_train, y_train, epochs=20, verbose=1);

Epoch 1/20
6/6 [==============================] - 11s 81ms/step - loss: 2.8127 - accuracy: 0.1829
Epoch 2/20
6/6 [==============================] - 0s 44ms/step - loss: 1.9084 - accuracy: 0.1951
Epoch 3/20
6/6 [==============================] - 0s 44ms/step - loss: 1.6591 - accuracy: 0.3598
Epoch 4/20
6/6 [==============================] - 0s 45ms/step - loss: 1.3873 - accuracy: 0.5549
Epoch 5/20
6/6 [==============================] - 0s 44ms/step - loss: 1.2012 - accuracy: 0.5122
Epoch 6/20
6/6 [==============================] - 0s 44ms/step - loss: 0.8184 - accuracy: 0.8293
Epoch 7/20
6/6 [==============================] - 0s 44ms/step - loss: 0.5171 - accuracy: 0.8841
Epoch 8/20
6/6 [==============================] - 0s 45ms/step - loss: 0.3425 - accuracy: 0.8963
Epoch 9/20
6/6 [==============================] - 0s 44ms/step - loss: 0.2177 - accuracy: 0.9390
Epoch 10/20
6/6 [==============================] - 0s 45ms/step - loss: 0.2207 - accuracy: 0.9329
Epoch 11/20
6/6 [===========

## 4. Exercise: Test the Model

In [ ]:
# Hints: You can refer to Lab05
# Exercise: Select the 7th image in the test set as sample, and pass it to the model to see the classification result

# Step 1: select 7-th image as sample


# Step 2: Print the original label for the image


# Step 3: Plot the image to validate


# Step 4: Predict sample image


# Step 5: Use np argmax to get the highest number (most activated) of output layer


# Step 6: Convert the prediction result to string



# Comparing Neural Network with CNN

In [ ]:
# train the basic neural network we previously have defined
model_nn.fit(x_train, y_train, epochs=40)

Epoch 1/40
6/6 [==============================] - 0s 8ms/step - loss: 19.4168 - accuracy: 0.1646
Epoch 2/40
6/6 [==============================] - 0s 8ms/step - loss: 12.1028 - accuracy: 0.1890
Epoch 3/40
6/6 [==============================] - 0s 8ms/step - loss: 7.3149 - accuracy: 0.1890
Epoch 4/40
6/6 [==============================] - 0s 8ms/step - loss: 3.7050 - accuracy: 0.2256
Epoch 5/40
6/6 [==============================] - 0s 9ms/step - loss: 2.1397 - accuracy: 0.2439
Epoch 6/40
6/6 [==============================] - 0s 8ms/step - loss: 1.9467 - accuracy: 0.3049
Epoch 7/40
6/6 [==============================] - 0s 7ms/step - loss: 1.8385 - accuracy: 0.3598
Epoch 8/40
6/6 [==============================] - 0s 8ms/step - loss: 1.7046 - accuracy: 0.3598
Epoch 9/40
6/6 [==============================] - 0s 8ms/step - loss: 1.6439 - accuracy: 0.3720
Epoch 10/40
6/6 [==============================] - 0s 8ms/step - loss: 1.8436 - accuracy: 0.3354
Epoch 11/40
6/6 [====================

In [ ]:
score_accuracy = model_nn.evaluate(x_test, y_test)

1/1 [==============================] - 0s 138ms/step - loss: 1.0247 - accuracy: 0.6316
